https://medium.com/@curiousily/tensorflow-for-hackers-part-ii-building-simple-neural-network-2d6779d2f91b

In [1]:
import pandas as pd
from math import floor
import tensorflow as tf
import numpy as np
from sklearn import preprocessing

C:\Users\Elias\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data1 = pd.read_csv('premier_14_15.csv')
data2 = pd.read_csv('premier_15_16.csv')
data3 = pd.read_csv('premier_16_17.csv')
data4 = pd.read_csv('premier_17_18.csv')

dataCon = [data1,data2,data3,data4]
data = pd.concat(dataCon)

C:\Users\Elias\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  import sys


In [3]:
b = np.random.uniform(low=-0.01,high=0.01,size=(3,3))
a = pd.DataFrame(b,columns=['a','b','c'])
a

,a,b,c
0,0.001212,-0.004858,-0.000829
1,0.008037,0.004584,-0.006322
2,0.005960,0.005987,-0.005590


In [4]:
def normalization(raw_data):
    for col_num in range(raw_data.shape[1]):
        if raw_data.iloc[:,col_num].dtype == np.float or raw_data.iloc[:,col_num].dtype == np.int:
            raw_data.iloc[:,col_num] = (raw_data.iloc[:,col_num] - raw_data.iloc[:,col_num].mean()) / (raw_data.iloc[:,col_num].max() - raw_data.iloc[:,col_num].min())
    return raw_data

In [5]:
def embedding_matrix(column):
    labels = []
    embeddings = np.array([])
    num_of_uniques = len(np.unique(column))
    for i in range(num_of_uniques):
        if embeddings.size == 0:
            embeddings = np.random.uniform(low=-0.01,high=0.01,size=(min(50,(num_of_uniques+1)//2),1))
        else:
            embeddings = np.append(embeddings,np.random.uniform(low=-0.01,high=0.01,size=(min(50,(num_of_uniques+1)//2),1)),axis=1)
        labels.append(np.unique(column)[i])
    print("embeddings.shape:",embeddings.shape)
    print("labels.shape:",len(labels))
    return pd.DataFrame(data=embeddings,columns=labels)

In [6]:
em = embedding_matrix(data['HomeTeam'])
em.head()

embeddings.shape: (13, 26)
labels.shape: 26


,Arsenal,Aston Villa,Bournemouth,Brighton,Burnley,Chelsea,Crystal Palace,Everton,Huddersfield,Hull,...,Norwich,QPR,Southampton,Stoke,Sunderland,Swansea,Tottenham,Watford,West Brom,West Ham
0,0.006524,-0.007546,-0.002282,-0.004149,0.002654,-0.006878,-0.008261,0.009751,0.003195,0.001499,...,-0.003738,-0.007296,0.008788,0.003515,0.004341,0.009036,-0.001634,-0.001646,0.003096,0.005937
1,-0.008860,0.001807,-0.009377,0.007840,-0.004247,0.003905,-0.004652,-0.002482,0.006690,-0.008111,...,0.000432,0.000952,0.009938,-0.008602,0.001691,-0.006902,0.007096,-0.001237,-0.005594,-0.006466
2,-0.006742,0.007364,-0.003975,-0.003631,-0.009697,0.009309,-0.009408,-0.000307,0.008672,-0.003531,...,-0.001095,-0.005577,0.009818,-0.009810,-0.007979,0.008825,0.007900,-0.005694,0.004628,0.001962
3,-0.005029,0.006436,-0.003642,0.003165,0.008926,0.004975,-0.008862,0.006278,-0.008563,-0.004011,...,-0.007517,0.003814,-0.002313,0.001819,0.009003,-0.004308,0.002752,0.003560,-0.002705,-0.006738
4,0.005114,0.006809,0.005715,0.003077,-0.009306,-0.002909,0.008079,-0.005622,0.000596,-0.004633,...,0.008663,0.005798,0.005405,-0.004258,0.003927,0.006883,-0.007978,0.007714,0.004417,0.008331


In [7]:
#x_data = data.drop(['FTR','FTAG','FTHG'],1)
data = normalization(data)
x_data = np.column_stack((np.transpose(em[data['HomeTeam']].values),
                          np.transpose(em[data['AwayTeam']].values),
                          data['B365H'][:,None],
                          data['B365D'][:,None],
                          data['B365A'][:,None]))
y_data = data['FTR']

y_data = pd.get_dummies(y_data)

#differences = data['FTHG']-data['FTAG']

In [8]:
x_data.shape

(1520, 29)

In [9]:
# Two first games
x_data[:,0:2]

array([[ 0.00652387, -0.00885993],
       [ 0.0005191 , -0.00142314],
       [ 0.00803522,  0.00487009],
       ...,
       [ 0.00903626, -0.00690173],
       [-0.00163394,  0.00709574],
       [ 0.00593708, -0.00646601]])

In [10]:
y_data.shape

(1520, 3)

In [11]:
y_data.head()

,A,D,H
0,0,0,1
1,0,1,0
2,1,0,0
3,1,0,0
4,1,0,0


In [12]:
train_size = 0.9
valid_size = 0.3

train_cnt = floor(x_data.shape[0] * train_size)

x_train = x_data[0:train_cnt]
y_train = y_data.iloc[0:train_cnt].values

valid_cnt = floor((x_data.shape[0] - train_cnt) * valid_size)

x_valid = x_data[train_cnt:train_cnt+valid_cnt]
y_valid = y_data.iloc[train_cnt:train_cnt+valid_cnt].values

x_test = x_data[train_cnt+valid_cnt:]
y_test = y_data.iloc[train_cnt+valid_cnt:]

print("x_train:",x_train.shape)
print("x_valid:",x_valid.shape)
print("x_test:",x_test.shape)

x_train: (1368, 29)
x_valid: (45, 29)
x_test: (107, 29)


In [13]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

In [24]:
# Parameters
LEARNING_RATE_STA = 7.5e-2
num_epochs = 30
batch_size = 128
display_step = 5

In [25]:
# Network Parameters
num_input = x_data.shape[1]
num_classes = y_data.shape[1]
num_hidden_1 = 50
num_hidden_2 = 50
num_hidden_3 = 50
KEEP_PROB_STA = 0.7

In [26]:
def neural_network(x,weights,biases,keep_prob):
    layer_1 = tf.add(tf.matmul(x,weights['w1']),biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1,keep_prob)
    
    layer_2 = tf.add(tf.matmul(layer_1,weights['w2']),biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.dropout(layer_2,keep_prob)
    
    layer_3 = tf.add(tf.matmul(layer_2,weights['w3']),biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    layer_3 = tf.nn.dropout(layer_3,keep_prob)
    
    layer_out = tf.add(tf.matmul(layer_3, weights['out']), biases['out'])
    return layer_out

In [27]:
# Store layers weight & bias
weights = {
    'w1': tf.Variable(tf.random_normal([num_input,num_hidden_1])),
    'w2': tf.Variable(tf.random_normal([num_hidden_1,num_hidden_2])),
    'w3': tf.Variable(tf.random_normal([num_hidden_2,num_hidden_3])),
    'out': tf.Variable(tf.random_normal([num_hidden_3, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'b3': tf.Variable(tf.random_normal([num_hidden_3])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

keep_prob = tf.placeholder("float")

In [28]:
predictions = neural_network(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE_STA).minimize(cost)

In [29]:
costs = []
lrn_rate_sizes = []

In [31]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: KEEP_PROB_STA
                            })
            avg_cost += c / total_batch
            
        if epoch % display_step == 0:
            print("Train: Epoch:", '%04d' % (epoch+display_step), "cost=", "{:.9f}".format(avg_cost))
            _, valid_cost = sess.run([optimizer, cost], feed_dict={x: x_valid, y: y_valid, keep_prob: 1})
            print("Valid: Epoch:", '%04d' % (epoch+display_step), "cost=", "{:.9f}".format(valid_cost))

            prediction=tf.argmax(predictions,1)
            print("Predictions:")
            print("")
            print("")
            print("")
            print(prediction.eval(feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))
            print("")
            print("")
            print("")
            print("$$$$$$$$$$$$$$$$$$$$$$$$")
            
            correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))
            print("______________________________________")
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))

Train: Epoch: 0005 cost= 88.096752357
Valid: Epoch: 0005 cost= 2.994447947
Predictions:



[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 1 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 2
 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2]



$$$$$$$$$$$$$$$$$$$$$$$$
Accuracy: 0.44859812
______________________________________
Train: Epoch: 0010 cost= 1.082212853
Valid: Epoch: 0010 cost= 1.004851699
Predictions:



[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]



$$$$$$$$$$$$$$$$$$$$$$$$
Accuracy: 0.45794392
______________________________________
Train: Epoch: 0015 cost= 1.060472327
Valid: Epoch: 0015 cost= 1.002155423
Predictions:



[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2